## WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [177]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import date
from scipy.stats import linregress
import json
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Range of latitudes and longitudes
lat_range = (-90.00, 90.00)
lng_range = (-180.00, 180.00)

# Generate Cities List

In [178]:
# Creates a function that generates randon coordinates, finds the nearest city of each coordinate and get the information 
# needed to the plots further in the code from the weather url

def find_information():
    
    def find_cities():

        coords = []
       
        #Generate a random list of 500+ cities
        lat = np.random.uniform(lat_range[0], lat_range[1], size=1500)
        long = np.random.uniform(lng_range[0], lng_range[1], size=1500)
        coords = list(zip(lat,long))


        for coord in coords:
            cityname = citipy.nearest_city(float(coord[0]), float(coord[1])).city_name
           
            #If the city is is not in the list of cities, then add the city to the list
            if cityname not in cities:
                cities.append(cityname)
        
        # Returns the result of the funcion find_cities, which is a list of cities that will be used for the API calls in the
        # function find_information
        return cities
         

    # to make sure that at least 650 cities will make to the api call
    if len('total_cities') <650:
        total_cities = find_cities()
 
    # Define variables
    response = []
    hum_list = []
    cl_list = []
    speed_list = []
    tmp_list = []
    lt_list = []
    lng_list = []
    ct_list = []
    country_list = []
    date_list = []
    
    # Base url
    base_url = 'http://api.openweathermap.org/data/2.5/weather?'    
    
    # query url for each city
    for city in total_cities:
        query_url = f"{base_url}&q={city}&units=imperial&id=Default&appid={weather_api_key}"
        
        #Get data and Jsonize it
        response = requests.get(query_url).json()
  
        try:
            # adds respective information for each list
            tmp_list.append(response["main"]["temp_max"])
            hum_list.append(response["main"]["humidity"])
            cl_list.append(response["clouds"]["all"])
            speed_list.append(response["wind"]["speed"])
            lt_list.append(response["coord"]["lat"])
            lng_list.append(response["coord"]["lon"])
            ct_list.append(response["name"])
            country_list.append(response["sys"]["country"])
            date_list.append(response["dt"])
            
        except:
            pass
    
    return tmp_list, hum_list, cl_list, speed_list, lt_list, lng_list, ct_list, country_list, date_list

In [179]:
cities = []
# Assigns the lists returned by the function to variables
temp_list, humidity_list, cloud_list, ws_list, lat_list, long_list, city_list, country, dates = find_information()

# Checks if the number of valid cities is at least 500. If it isn't it calls the function again to generate new outputs
if len(temp_list) <500:
    temp_list = []
    humidity_list = []
    cloud_list = []
    ws_list = []
    lat_list = []
    long_list =[]
    city_list = []
    country = []
    dates = []
    find_information()
print(len(temp_list))

568


# Convert Raw Data to DataFrame
Export the city data into a .csv.
Display the DataFrame# 

In [191]:
#city_ds =( pd.Series(city_list)).astype(str)
#lat_ds = (pd.Series(lat_list)).astype(object)
#lng_ds = (pd.Series(long_list)).astype(object)
#max_temp_ds = (pd.Series(temp_list)).astype(object)
#humd_ds = (pd.Series(humidity_list)).astype(object)
#cloud_ds = (pd.Series(cloud_list)).astype(object)
#ws_ds = (pd.Series(ws_list)).astype(object)
#country_ds = (pd.Series(country)).astype(object)
#date_ds = (pd.Series(dates)).astype(object)
#info_df = []

#Name columns of the dataframe
#info_df = pd.concat([city_ds, lat_ds,lng_ds,max_temp_ds,humd_ds,cloud_ds, ws_ds, country_ds,date_ds], axis=1)
info_df =pd.DataFrame [city_list,lat_list,long_list,temp_list,humidity_list,cloud_list,ws_list,country,dates]
info_df = info_df.rename(columns = {info_df.columns[0]:'City',info_df.columns[1]:'Lat',
                                                   info_df.columns[2]:'Lng', info_df.columns[3] : 'Max Temp',
                                                   info_df.columns[4] : 'Humidity',
                                                   info_df.columns[5] : 'Cloudiness',
                                                   info_df.columns[6] : 'Wind Speed',
                                                   info_df.columns[7] : 'Country',
                                                   info_df.columns[8] : 'Date'})

info_df.head(10)

TypeError: 'type' object is not subscriptable

In [169]:
info_df.describe()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
count,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
#Re-order columns and export the city data into a .csv. 


output_data_file = info_df[['City','Cloudiness','Date','Humidity','Lat','Lng','Max Temp','Wind Speed']]
output_data_file.to_csv('output_data/cities.csv', index_label='City ID', index=True, header=True)


# Inspect the data and remove the cities where the humidity > 100%

In [155]:
#  Get the indices of cities that have humidity over 100%.

index_value = output_data_file[output_data_file['Humidity']>100].index

if len(index_value) > 0:
    clean_city_data = output_data_file.drop(index=output_data_file.index[index_value])
    clean_city_data.head()
    

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

# Latitude vs. Temperature Plot 

In [156]:
today = today.strftime("%m/%d/%y")

AttributeError: 'str' object has no attribute 'strftime'

In [ ]:
plt.scatter(lat_list, temp_list, color='r')
plt.xlabel('Latitude')
plt.ylabel('Temperature(F)')
plt.title('City Latitude vs. max. Temperature ' + str(today))
plt.savefig('output_data/temp_lat.png')
plt.show()

# Latitude vs. Humidity Plot

In [ ]:
plt.scatter(lat_list, humidity_list, color='green')
plt.xlabel('Latitude')
plt.ylabel('Humidity %')
plt.title('City Latitude vs. Humidity ' + str(today))
plt.savefig('output_data/humidity_lat.png')
plt.show()

# Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(lat_list, cloud_list, color='blue')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness %')
plt.title('City Latitude vs. Cloudiness ' + str(today))
plt.savefig('output_data/cloudiness_lat.png')
plt.show()

# Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(lat_list, ws_list, color='orange')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('City Latitude vs. Wind Speed ' + str(today))
plt.savefig('output_data/ws_lat.png')
plt.show()

# Linear Regression

In [ ]:
# Split data by hemisphere

hem_north = []
temp_north = []
humidity_north = []
cloudiness_north = []
ws_north = []
hem_south = []
temp_south = []
humidity_south = []
cloudiness_south = []
ws_south = []

for i in range(len(lat_list)):
    if (lat_list[i]) >= 0.0:
        hem_north.append(lat_list[i])
        temp_north.append(temp_list[i])
        humidity_north.append(humidity_list[i])
        cloudiness_north.append(cloud_list[i])
        ws_north.append(ws_list[i])
    else:
        hem_south.append(lat_list[i])
        temp_south.append(temp_list[i])
        humidity_south.append(humidity_list[i])
        cloudiness_south.append(cloud_list[i])
        ws_south.append(ws_list[i])        


###  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Temp plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_north,temp_north)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_north) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_north,temp_north,  color = 'lightblue')
plt.plot(hem_north,regress_values,"black")
plt.annotate(line_eq,(10,-40),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Max Temp(F)')
plt.xlim(0,90)
plt.ylim(-60,120)
plt.savefig('output_data/temp_north.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between temperature and latitude in the north hemisphere.')
else:
    print('The plot above does not show any linear correlation between temperature and latitude in the north hemisphere.')
print('')


###  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Temp plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_south,temp_south)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_south) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_south,temp_south,  color = 'darkblue')
plt.plot(hem_south,regress_values,"black")
plt.annotate(line_eq,(-80,-40),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Max Temp(F)')
plt.xlim(-90,0)
plt.ylim(-60,120)
plt.savefig('output_data/temp_south.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between temperature and latitude in the south hemisphere.')
else:
    print('The plot above does not show any linear correlation between temperature and latitude in the south hemisphere.')
print('')

###  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Humidity plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_north,humidity_north)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_north) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_north,humidity_north,  color = 'lightgreen')
plt.plot(hem_north,regress_values,"black")
plt.annotate(line_eq, (50,25),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Humidity %')
plt.xlim(0,90)
plt.savefig('output_data/humidity_north.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between humidity and latitude in the north hemisphere.')
else:
    print('The plot above does not show any linear correlation between humidity and latitude in the north hemisphere.')
print('')

###  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Humidity plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_south,humidity_south)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_south) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_south,humidity_south,  color = 'green')
plt.plot(hem_south,regress_values,"black")
plt.annotate(line_eq, (-80,50),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Humidity %')
plt.xlim(-90,0)
plt.savefig('output_data/humidity_south.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between humidity and latitude in the south hemisphere.')
else:
    print('The plot above does not show any linear correlation between humidity and latitude in the south hemisphere.')
print('')

###  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Cloudiness plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_north,cloudiness_north)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_north) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_north,cloudiness_north,  color = 'red')
plt.plot(hem_north,regress_values,"black")
plt.annotate(line_eq, (50,25),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness %')
plt.xlim(0,90)
plt.savefig('output_data/cloudiness_north.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between percentage of clouds and latitude in the north hemisphere.')
else:
    print('The plot above does not show any linear correlation between percentage of clouds and latitude in the north hemisphere.')
print('')

### Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Cloudiness plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_south,cloudiness_south)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_south) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_south,cloudiness_south,  color = 'magenta')
plt.plot(hem_south,regress_values,"black")
plt.annotate(line_eq, (-80,20),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness %')
plt.xlim(-90,0)
plt.savefig('output_data/cloudiness_south.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between percentage of clouds and latitude in the south hemisphere.')
else:
    print('The plot above does not show any linear correlation between percentage of clouds and latitude in the south hemisphere.')
print('')

###  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Wind speed plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_north,ws_north)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_north) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_north,ws_north,  color = 'grey')
plt.plot(hem_north,regress_values,"black")
plt.annotate(line_eq, xy=(10, 12),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Wind speed [mph]')
plt.xlim(0,90)
plt.savefig('output_data/ws_north.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between wind speed and latitude in the north hemisphere.')
else:
    print('The plot above does not show any linear correlation between wind speed and latitude in the north hemisphere.')
print('')

###  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Wind speed plots
regress_values=[]
(slope, intercept, rvalue, pvalue, stderr) = linregress(hem_south,ws_south)
print("R-square = " + str(round(rvalue **2,2)))
regress_values = np.asarray(hem_south) * slope + intercept
line_eq = "y = " + str(round(slope,3)) + "x + " + str(round(intercept,3))
plt.scatter(hem_south,ws_south,  color = 'orange')
plt.plot(hem_south,regress_values,"black")
plt.annotate(line_eq, (-80,12),fontsize=12,color="black")
plt.xlabel('Latitude')
plt.ylabel('Wind speed [mph]')
plt.xlim(-90,0)
plt.savefig('output_data/ws_south.png')
plt.show()
if (rvalue**2)> 0.6:
    print('The plot above shows a linear correlation between wind speed and latitude in the south hemisphere.')
else:
    print('The plot above does not show any linear correlation between wind speed and latitude in the south hemisphere.')
print('')